In [10]:
print( 'ФИО: Кирилл Живин' )

import os
import logging
import psycopg2
import psycopg2.extensions
from pymongo import MongoClient

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

logger.info("Создаём подключёние к Postgres")
params = {
    "host": os.environ['APP_POSTGRES_HOST'],
    "port": os.environ['APP_POSTGRES_PORT'],
    "user": 'postgres'
}
conn = psycopg2.connect(**params)

psycopg2.extensions.register_type(
    psycopg2.extensions.UNICODE,
    conn
)
conn.set_isolation_level(
    psycopg2.extensions.ISOLATION_LEVEL_AUTOCOMMIT
)

import datetime
from subprocess import call
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)


## запрос 3.1 : получаем среднюю стоимость заказа

df = pd.read_sql_query("select*from orders;", conn)
result = df. groupby([True]*len(df)).apply(lambda x: x['order_cart_sum'].sum()/len(x))

print(result)


## запрос 3.2 : получаем кол-во заказов > 10 пирогов

df = pd.read_sql_query("select*from order_cart;", conn)
df = df[df['pie_count']>10]
result = df.groupby([True]*len(df)).apply(lambda x: len(x))

print(result) 


## запрос 3.3 : получаем заказы оформленные online и со стоимостью доставки >= 500

df = pd.read_sql_query("select*from orders;", conn)
price_condition = df['order_delivery_price']>=500
source_condition = df['order_source']=='internet'
df = df[price_condition & source_condition]
result = df['order_id'].head(10)

print(result)


## запрос 3.9 : получаем среднюю стоймость заказа в зависимости от стоймости доставки

df = pd.read_sql_query("select*from orders;", conn)
delivery_condition = df['order_delivery_price']!=0
df = df[delivery_condition]
cart_sum = df['cart_sum'] = df.groupby('order_delivery_price').apply(lambda x: x['order_cart_sum'].sum()/len(x))
print(cart_sum)


## запрос 3.10 : получаем среднее кол-во пирогов в заказе у заказчиков с историей > = 5 заказов

df1 = pd.read_sql_query("select*from order_persons;", conn)
a1 = df1.groupby('person_id').filter(lambda x: len(x)>= 5).drop_duplicates('person_id')

df2 = pd.read_sql_query("select*from order_cart;", conn)
order_persons = pd.merge(a1, df1, on='person_id', how='inner')

a2 = pd.merge(order_persons, df2, left_on='order_id_y', right_on='cart_order_id', how='inner')
r = a2.groupby('cart_order_id').apply(lambda x: pd.Series({'sum_pie': x['pie_count'].sum() }))

result = r.groupby([True]*len(r)).apply(lambda x: x['sum_pie'].sum() / len(x))

print(result)

df.to_csv("/home/pie.csv")




ФИО: Кирилл Живин


ModuleNotFoundError: No module named 'psycopg2'